In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import math
import re


In [3]:
train = pd.read_csv("train.csv")


In [4]:
test = pd.read_csv("test.csv")
submission = pd.read_csv("sample_submission.csv")

시작시간 없는건 최빈값으로 채우기

두 컬럼 둘다 fillna(0) 해주기.

분으로 바꿔서 총 운항시간(분) 확인하기

distance랑 비교해서 값내고 1분에 몇마일 or 1마일에 몇분 컬럼 추가

모든 컬럼값 더해서 평균내기

평균낸걸로 distance 비교해서 채우기



In [5]:
count = train['Estimated_Arrival_Time'].value_counts().get(600.0, 0)
print(count)

483


In [6]:
count = train['Estimated_Departure_Time'].value_counts().get(600.0, 0)
print(count)


17423


In [7]:
for index, row in train.iterrows():
    if pd.isnull(row['Estimated_Arrival_Time']) and pd.isnull(row['Estimated_Departure_Time']):
        train.at[index, 'Estimated_Departure_Time'] = train['Estimated_Departure_Time'].mode().iloc[0]


In [11]:
train['Estimated_Departure_Time'].isnull().sum()

0

In [12]:
train['Estimated_Arrival_Time'].isnull().sum()

0

In [54]:
null_mask = train[['Estimated_Departure_Time', 'Estimated_Arrival_Time']].isnull().all(axis=1)
null_rows = train[null_mask]

print(f"{len(null_rows)}")


0


In [10]:
train['Estimated_Departure_Time'] = train['Estimated_Departure_Time'].fillna(0)
train['Estimated_Departure_Time'] = train['Estimated_Departure_Time'].astype(int)

train['Estimated_Arrival_Time'] = train['Estimated_Arrival_Time'].fillna(0)
train['Estimated_Arrival_Time'] = train['Estimated_Arrival_Time'].astype(int)

In [19]:
def convert_to_time(minutes):
    hours = minutes // 60
    minutes = minutes % 60
    return f"{hours:02d}{minutes:02d}"

In [ ]:
def convert_to_minutes(time):    
    return int(time // 100) * 60 + int(time % 100)

In [37]:
train['Flight_Time'] = None
train['Estimated_Departure_Minutes'] = None
train['Estimated_Arrival_Minutes'] = None

for index, row in train.iterrows():
    dep_time = row['Estimated_Departure_Time']
    arr_time = row['Estimated_Arrival_Time']
    
    if arr_time == 0 and dep_time == 0:
        continue
        
    elif dep_time == 0:
        arr_time_in_minutes = convert_to_minutes(arr_time)
        train.at[index, 'Estimated_Arrival_Minutes'] = arr_time_in_minutes
        
    elif arr_time == 0:
        dep_time_in_minutes = convert_to_minutes(dep_time)
        train.at[index, 'Estimated_Departure_Minutes'] = dep_time_in_minutes
        
    else:
        dep_time_in_minutes = convert_to_minutes(dep_time)
        arr_time_in_minutes = convert_to_minutes(arr_time)
        
        if arr_time_in_minutes < dep_time_in_minutes:
            arr_time_in_minutes += 1440
        
        duration_in_minutes = arr_time_in_minutes - dep_time_in_minutes
        train.at[index, 'Flight_Time'] = duration_in_minutes
        train.at[index, 'Estimated_Arrival_Minutes'] = arr_time_in_minutes
        train.at[index, 'Estimated_Departure_Minutes'] = dep_time_in_minutes


In [73]:
train

,ID,Month,Day_of_Month,Estimated_Departure_Time,Estimated_Arrival_Time,Cancelled,Diverted,Origin_Airport,Origin_Airport_ID,Origin_State,...,Distance,Airline,Carrier_Code(IATA),Carrier_ID(DOT),Tail_Number,Delay,Flight_Time,Estimated_Departure_Minutes,Estimated_Arrival_Minutes,Flight_Time_per_Distance
0,TRAIN_000000,4,15,600,0,0,0,OKC,13851,Oklahoma,...,419.0,Southwest Airlines Co.,WN,19393.0,N7858A,NaN,97.601139,360,None,NaN
1,TRAIN_000001,8,15,740,1024,0,0,ORD,13930,Illinois,...,1250.0,SkyWest Airlines Inc.,UA,20304.0,N125SY,NaN,164,460,624,0.131200
2,TRAIN_000002,9,6,1610,1805,0,0,CLT,11057,North Carolina,...,544.0,American Airlines Inc.,AA,19805.0,N103US,NaN,115,970,1085,0.211397
3,TRAIN_000003,7,10,905,1735,0,0,LAX,12892,California,...,2454.0,United Air Lines Inc.,UA,NaN,N595UA,NaN,510,545,1055,0.207824
4,TRAIN_000004,1,11,900,1019,0,0,SFO,14771,California,...,250.0,SkyWest Airlines Inc.,UA,20304.0,N161SY,NaN,79,540,619,0.316000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999995,TRAIN_999995,9,18,936,1243,0,0,ORD,13930,NaN,...,678.0,United Air Lines Inc.,UA,19977.0,N477UA,NaN,187,576,763,0.275811
999996,TRAIN_999996,5,30,920,1028,0,0,FAR,11637,NaN,...,223.0,SkyWest Airlines Inc.,DL,NaN,N439SW,NaN,68,560,628,0.304933
999997,TRAIN_999997,6,28,800,1340,0,0,OAK,13796,NaN,...,1642.0,Southwest Airlines Co.,WN,19393.0,N230WN,NaN,340,480,820,0.207065
999998,TRAIN_999998,9,27,1613,1824,0,0,BNA,10693,Tennessee,...,214.0,Delta Air Lines Inc.,DL,19790.0,N968DL,NaN,131,973,1104,0.612150


In [41]:
for index, row in train.iterrows():
    if pd.isnull(row['Flight_Time']):
        train.at[index, 'Flight_Time_per_Distance'] = None
    else:
        flight_time_per_distance = row['Flight_Time'] / row['Distance']
        train.at[index, 'Flight_Time_per_Distance'] = flight_time_per_distance

In [42]:
avg_list = []
for index, row in train.iterrows():
    if pd.isnull(row['Flight_Time_per_Distance']):
        pass
    else:
        avg_list.append(row['Flight_Time_per_Distance'])
 

In [49]:
print(sum(avg_list), len(avg_list))

184866.57368187737 793629


In [51]:
velocity_avg = sum(avg_list) / len(avg_list)
velocity_avg

0.23293827932431574

In [53]:
t_f = []
for i in avg_list:
    if i > velocity_avg:
        t_f.append('big')
    else:
        t_f.append('small')

In [56]:
print(t_f.count('small'), t_f.count('big'))

500573 293056


In [68]:
for index, row in train.iterrows():
    if row['Flight_Time'] is None:
        train.at[index, 'Flight_Time'] = row['Distance'] * velocity_avg
    else:
        pass

In [74]:
train['Flight_Time'].isnull().sum()

0